# ***Import Libraries***

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
from tqdm import tqdm
import datasets
import torch
import os
import pickle
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer,EarlyStoppingCallback
import warnings
warnings.filterwarnings('ignore')

# ***configuration***

In [5]:
TEXT_MAX_TOKEN_LENGTH = 512
SUMMARY_MAX_TOKEN_LENGTH = 192

# ***Data Reading***

In [6]:
dataset = datasets.load_dataset("csebuetnlp/xlsum", name="arabic")

README.md:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

xlsum.py:   0%|          | 0.00/4.55k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/95.1M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/10.6M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/10.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/37519 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4689 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4689 [00:00<?, ? examples/s]

In [7]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 37519
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4689
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4689
    })
})


# ***Split Data***

In [8]:
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

In [9]:
train_df.head()

,id,url,title,summary,text
0,140323_russian_troops_crimea_naval_base,https://www.bbc.com/arabic/worldnews/2014/03/1...,القوات الأوكرانية تبدأ الانسحاب من القرم,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة ا...,وكان الرئيس الأوكراني المؤقت، الكسندر تورتشينو...
1,130528_egypt_nile_dam,https://www.bbc.com/arabic/middleeast/2013/05/...,هل يفرض سد النهضة الإثيوبي واقعا جديدا على مصر؟,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...",بحلول عام 2050 ستحتاج مصر إلى 21 مليار متر مكع...
2,world-47242349,https://www.bbc.com/arabic/world-47242349,تعرف على منطقة كشمير التي تسببت بحربين بين اله...,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...,وذكرت وكالة الأنباء المحلية (جي.إن.إس) أن جماع...
3,vert-cul-55078328,https://www.bbc.com/arabic/vert-cul-55078328,ماذا تعرف عن العالم الخفي للمعابد اليابانية ال...,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...,ووقع اختياره على واد عمقه 800 متر محاط بثماني ...
4,141023_yemen_hodeida,https://www.bbc.com/arabic/middleeast/2014/10/...,"اشتباك بين الحوثيين و""الحراك التهامي"" في الحدي...","أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح...",مسلح حوثي في إب وقال المصدر إن المسلحين الحوثي...


In [10]:
valid_df.head()

,id,url,title,summary,text
0,140921_yemen_pm_resign,https://www.bbc.com/arabic/middleeast/2014/09/...,اتفاق لتشكيل حكومة جديدة بين الحوثيين وعدة احز...,وقع الحوثيون اتفاقا مع عدد من الاحزاب اليمنية ...,انضم بعض عناصر القوات المسلحة الى الحوثيين في ...
1,inthepress-56197077,https://www.bbc.com/arabic/inthepress-56197077,مظاهرات تونس: إلى أي مدى سيصل الصراع بين أطراف...,ناقش معلقون في صحف عربية تطور الأزمة السياسية ...,الرئيس التونسي قيس سعيد ورئيس الحكومة هشام الم...
2,151114_gatwick_airport_frenchman_charged,https://www.bbc.com/arabic/worldnews/2015/11/1...,اتهام رجل بحيازة أسلحة بعد إعلان حالة تأهب في ...,اتهمت الشرطة البريطانية رسميا رجلا فرنسيا بحيا...,المسافرون انتظروا ست ساعات على أرض مطار غاتويك...
3,media-48880286,https://www.bbc.com/arabic/media-48880286,عالم سري للسوريّات على فيسبوك محظور على الرجال...,تطلب شابة على إحدى صفحات فيسبوك المغلقة نصيحة ...,وانهالت تعليقات النساء على هذه المداخلة بين مؤ...
4,130311_us_nkorea,https://www.bbc.com/arabic/middleeast/2013/03/...,الولايات المتحدة توسع العقوبات ضد كوريا الشمالية,فرضت الولايات المتحدة عقوبات على بنك العملة ال...,قطعت كوريا الشمالية الخط الهاتفي الساخن بين ال...


In [11]:
test_df.head()

,id,url,title,summary,text
0,130806_nidhal_hassan_trial,https://www.bbc.com/arabic/worldnews/2013/08/1...,نضال حسن يمثل أمام محكمة عسكرية أمريكية,تنظر محكمة عسكرية أمريكية في وقت لاحق من اليوم...,نضال حسن واعترف نضال حسن، الذي يدافع عن نفسه، ...
1,160129_germany_asylum_seekers,https://www.bbc.com/arabic/worldnews/2016/01/1...,ألمانيا تسعى للحد من الهجرة بإعلان الجزائر وتو...,كشفت ألمانيا النقاب عن خطط لإضافة الجزائر والم...,ألمانيا تواجه مصاعب في التعامل مع الأعداد المت...
2,130729_syria_homs_area_retaken,https://www.bbc.com/arabic/middleeast/2013/07/...,الخالدية : التليفزيون السوري يعلن استعادة الجي...,قال التليفزيون السوري إن قوات الحكومة استعادت ...,"وأكدت وسائل الإعلام السورية أن الجيش ""استعاد ا..."
3,140517_arsenal_fa_cup_winners,https://www.bbc.com/arabic/sports/2014/05/1405...,الارسنال يتوج بلقب كأس انكلترا,توج فريق الارسنال ببطولة كأس انجلترا لكرة القد...,وفاجأ هال سيتي الحضور بمباغتة الارسنال بهدفين ...
4,140722_iraq_minorities,https://www.bbc.com/arabic/middleeast/2014/07/...,العراق: الأقليات في سهل نينوى,يوضع الصراع في العراق غالبا في إطار صراع بين ا...,معبد يزيدي في سهول محافظة نينوى يعيش المسيحيون...


In [12]:
train_df.shape, valid_df.shape, test_df.shape

((37519, 5), (4689, 5), (4689, 5))

# ***Load Model***

In [13]:
MODEL_NAME = "UBC-NLP/AraT5v2-base-1024"

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)

tokenizer_config.json:   0%|          | 0.00/21.1k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# ***Dataset***

In [16]:
class TextSummarizationDataset(torch.utils.data.Dataset):
  def __init__(self, dataframe, tokenizer, text_max_token_length = TEXT_MAX_TOKEN_LENGTH, summary_max_token_length=SUMMARY_MAX_TOKEN_LENGTH):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.text_max_token_length = text_max_token_length
    self.summary_max_token_length = summary_max_token_length

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    text = data_row['text']
    summary = data_row['summary']
    text_encoding = tokenizer(text, max_length=self.text_max_token_length, padding='max_length', truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors='pt')
    summary_encoding = tokenizer(summary, max_length=self.summary_max_token_length, padding='max_length', truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors='pt')
    labels = summary_encoding['input_ids']
    labels[labels == self.tokenizer.pad_token_id] = -100

    return dict(
        input_ids=text_encoding['input_ids'].flatten(),
        attention_mask=text_encoding['attention_mask'].flatten(),
        labels=labels.flatten()
    )


In [17]:
train_dataset = TextSummarizationDataset(train_df, tokenizer)
valid_dataset = TextSummarizationDataset(valid_df, tokenizer)

In [18]:
train_dataset[0]

{'input_ids': tensor([  2163,   1413,  68318,    379,  31337, 109598,  79397,  15573,   7140,
             38,    336, 109598,    566,   2873,  47353,   1859,   4783,  68318,
            771,     36,  76147, 109566,   4659,  21764,   4465,  43357,   1257,
           4995,    114,  10250,  30285,    477,    210,    771,     43,     43,
            218,   5218, 109598,     43,  12790,  10555,     36,  37511,    821,
          10798,   3581, 109598,   2500,   9916,   8447,  26998,    477,    210,
          11533,  35888,    600,    728,    180,   1110,   8457,   3506,   2443,
           4096,  68318,    771,   1908,  19795,   2118,    728,     60,  21157,
            336,    364,   4783,  11634,  90836,  10460,  12394,    363,   8888,
            114,  16858,  68318,  11533,     43,  10250,     43,    218,   5218,
            467,    189, 109551,   9851,    112,  19899,    116, 109566,    661,
          12666,    141,  30041,   9413,  68318,    771,    114,   6649,   4263,
           1606

# ***Metrics for evaluation***

In [32]:
!pip install evaluate rouge_score bert_score
clear_output()

In [33]:
import evaluate
rouge_metric = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
clear_output()

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    vocab_size = len(tokenizer.get_vocab())
    predictions = np.clip(predictions, 0, vocab_size - 1)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)


    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(pred.split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.split()) for label in decoded_labels]


    result = rouge_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True 
    )

    result = {key: value * 100 for key, value in result.items()}
    
    return {k: round(v, 4) for k, v in result.items()}

# ***Training***

In [20]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="output-AraT5v2-XLSum-arabic-summarizer",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=10,  
    logging_strategy="epoch",  
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    remove_unused_columns=False,
    label_names=["labels"],
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    push_to_hub=False,

    warmup_ratio=0.1, 
    lr_scheduler_type="cosine",
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [22]:
callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=callbacks,
)

In [24]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.999300,2.415354,2.198000,0.106600,2.222500,2.199700
2,2.879100,2.392637,2.416900,0.103100,2.427800,2.410800
3,2.646900,2.324992,2.354500,0.106600,2.370000,2.358200
4,2.472900,2.312325,2.617200,0.120900,2.633000,2.613400
5,2.388600,2.314750,2.675200,0.120900,2.694700,2.665700


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=46900, training_loss=2.877349205423774, metrics={'train_runtime': 31788.2086, 'train_samples_per_second': 5.901, 'train_steps_per_second': 1.475, 'total_flos': 1.6301464928649216e+17, 'train_loss': 2.877349205423774, 'epoch': 5.0})

# ***Evaluate The Model***

In [25]:
trainer.evaluate()

{'eval_loss': 2.3123252391815186,
 'eval_rouge1': 2.6172,
 'eval_rouge2': 0.1209,
 'eval_rougeL': 2.633,
 'eval_rougeLsum': 2.6134,
 'eval_runtime': 942.1597,
 'eval_samples_per_second': 4.977,
 'eval_steps_per_second': 1.245,
 'epoch': 5.0}

# ***Save Model***

In [ ]:
model_save_path = "AraT5v2-XLSum-arabic-summarizer"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to AraT5v2-XLSum-arabic-summarizer


# ***Load Model***

In [27]:
# model_path = "/kaggle/input/text-summarization/Ara-Bart-XLSum-arabic-summarizer"

# model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
# tokenizer = AutoTokenizer.from_pretrained(model_path)


# ***Evaluation on Test Set***

In [22]:
test_df.head()

,id,url,title,summary,text
0,130806_nidhal_hassan_trial,https://www.bbc.com/arabic/worldnews/2013/08/1...,نضال حسن يمثل أمام محكمة عسكرية أمريكية,تنظر محكمة عسكرية أمريكية في وقت لاحق من اليوم...,نضال حسن واعترف نضال حسن، الذي يدافع عن نفسه، ...
1,160129_germany_asylum_seekers,https://www.bbc.com/arabic/worldnews/2016/01/1...,ألمانيا تسعى للحد من الهجرة بإعلان الجزائر وتو...,كشفت ألمانيا النقاب عن خطط لإضافة الجزائر والم...,ألمانيا تواجه مصاعب في التعامل مع الأعداد المت...
2,130729_syria_homs_area_retaken,https://www.bbc.com/arabic/middleeast/2013/07/...,الخالدية : التليفزيون السوري يعلن استعادة الجي...,قال التليفزيون السوري إن قوات الحكومة استعادت ...,"وأكدت وسائل الإعلام السورية أن الجيش ""استعاد ا..."
3,140517_arsenal_fa_cup_winners,https://www.bbc.com/arabic/sports/2014/05/1405...,الارسنال يتوج بلقب كأس انكلترا,توج فريق الارسنال ببطولة كأس انجلترا لكرة القد...,وفاجأ هال سيتي الحضور بمباغتة الارسنال بهدفين ...
4,140722_iraq_minorities,https://www.bbc.com/arabic/middleeast/2014/07/...,العراق: الأقليات في سهل نينوى,يوضع الصراع في العراق غالبا في إطار صراع بين ا...,معبد يزيدي في سهول محافظة نينوى يعيش المسيحيون...


In [23]:
len(test_df)

4689

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples,
        padding="max_length",
        truncation=True
        max_length=TEXT_MAX_TOKEN_LENGTH,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)

    outputs = model.generate(
        input_ids, 
        attention_mask=attention_mask,
        max_length=SUMMARY_MAX_TOKEN_LENGTH,
        min_length=10,   
        num_beams=4,
        repetition_penalty=2.0,
        length_penalty=1.0,
        no_repeat_ngram_size = 2
    )

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    return output_str

In [35]:
predictions = []
for i in tqdm(range(len(test_df))):
    input_text = test_df['text'].iloc[i]
    generated_summary = generate_summary(input_text, model)
    predictions.append(generated_summary)


100%|██████████| 4689/4689 [47:17<00:00,  1.65it/s]


In [37]:
bertscore_results = bertscore.compute(
    predictions=predictions,
    references=test_df['summary'],
    lang="ar"
)

print("\n BERTScore Results:")
print(f"Average Precision: {sum(bertscore_results['precision']) / len(bertscore_results['precision']):.4f}")
print(f"Average Recall:    {sum(bertscore_results['recall']) / len(bertscore_results['recall']):.4f}")
print(f"Average F1 Score:  {sum(bertscore_results['f1']) / len(bertscore_results['f1']):.4f}")



 BERTScore Results:
Average Precision: 0.7871
Average Recall:    0.7603
Average F1 Score:  0.7730


# ***Inference***

In [27]:
# text = "شهدت مدينة طرابلس، مساء أمس الأربعاء، احتجاجات شعبية وأعمال شغب لليوم الثالث على التوالي، وذلك بسبب تردي الوضع المعيشي والاقتصادي. واندلعت مواجهات عنيفة وعمليات كر وفر ما بين الجيش اللبناني والمحتجين استمرت لساعات، إثر محاولة فتح الطرقات المقطوعة، ما أدى إلى إصابة العشرات من الطرفين."

In [28]:
summary = generate_summary(text, model)
print(summary)

شهدت مدينة طرابلس اللبنانية، مساء أمس الأربعاء، احتجاجات شعبية وأعمال شغب لليوم الثالث على التوالي، وذلك بسبب تردي الوضع المعيشي.


In [70]:
for i in range(5):
    random_row = test_df.sample(n=1).iloc[0]
    
    input_text = random_row['text']
    target_text = random_row['summary']
    
    print("📝 Text before summarization:\n", input_text)
    
    print("\n✅ Original (Reference) Summary:\n", target_text)
    
    generated_summary = generate_summary(input_text, model)
    print("\n🤖 Model-generated Summary:\n", generated_summary)
    print("----------------------------------------------------")


📝 Text before summarization:
 وأصيب النجم المصري بكدمة في الرأس عندما كان يحاول الوصول إلى كرة عالية في منطقة الجزاء اشترك فيها حارس مرمى نيوكاسل مارتن دوبرافكا في الدقيقة 68 قبل أن يقع صلاح على رأسه. وتترك هذه الإصابة مشاركة صلاح في مباراة العودة لقبل نهائي دوري أبطال أوروبا أمام برشلونة الثلاثاء المقبل موضع شك. وكان صلاح في وعيه أثناء خروجة وكان يتحدث لطبيب ليفربول ما يشير إلى إمكانية أن تكون الإصابة بسيطة. وتمكن ليفربول من الفوز بثلاثة أهداف على مضيفه مقابل هدفين سجل فيرجيل فان دايك الهدف الأول وصلاح الهدف الثاني و ديفوك أوريغي الهدف الثالث قبل نهاية الوقت الأصلي بثلاث دقائق. وبهذه النتيجة رفع ليفربول رصيده إلى 94 نقطة في صدارة البطولة مؤقتا حتى يلتقي مانشستر سيتي ليستر سيتي الإثنين ضمن مباريات الجولة نفسها وفي حال فوزه سيرفع رصيده إلى 95 نقطة قبل مباراة واحدة من نهاية البطولة. وهذا الهدف هو رقم مائة في رصيد صلاح في الدوريات الأوروبية، كما رفع رصيده إلى 22 هدفا في صدارة هدافي البريميير ليغ هذا الموسم بفارق هدفين عن سيرجيو أغويرو وساديو ماني.

✅ Original (Reference) Summary:
 تعرض ال

In [32]:
from huggingface_hub import notebook_login

# Login to Hugging Face
notebook_login()

In [33]:
model.push_to_hub("omarsabri8756/AraT5v2-XLSum-arabic-text-summarization")
tokenizer.push_to_hub("omarsabri8756/AraT5v2-XLSum-arabic-text-summarization")

model.safetensors:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/omarsabri8756/AraT5v2-XLSum-arabic-text-summarization/commit/693b9ecd01e953457147c392127167cadf3dfb6c', commit_message='Upload tokenizer', commit_description='', oid='693b9ecd01e953457147c392127167cadf3dfb6c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/omarsabri8756/AraT5v2-XLSum-arabic-text-summarization', endpoint='https://huggingface.co', repo_type='model', repo_id='omarsabri8756/AraT5v2-XLSum-arabic-text-summarization'), pr_revision=None, pr_num=None)